In [1]:
import numpy as np
import pandas as pd
import plotnine as gg

In [2]:
def clean_column_name(column_name):
    return (
        column_name
        .strip()
        .lower()
        .replace(' ', '_')
        .replace('-', '_')
        .replace('\n', '_')
        .replace('(', '')
        .replace(')', '')
        .replace('%', 'pecent')
        .replace('/', '_or_')
        .replace('?', '')
        .replace('__', '_')
        .replace('_dates', '_date')
    )

dict_manual_renamings = {
    'campaign_symbol': 'campaign_id',
    'activist': 'activist_name',
    'campaign_announce_date': 'campaign_announcement_date',
    'in_force_prior_to_announcement_poison_pill': 'poison_pill_in_force_prior_to_announcement',
    'adopted_in_response_to_campaign_poison_pill': 'poison_pill_adopted_in_response_to_campaign',
}

list_drop_columns = [
    
    # repeat of campaign_announce_date
    'announcement_date_date', 
    
    # unused for now
    '18_months_pre_date_stock_price',
    '1_year_pre_date_stock_price',
    '6_months_pre_date_stock_price',
    '90_days_pre_date_stock_price',
    
    '18_months_pre_date_dividends',
    '1_year_pre_date_dividends',
    '6_months_pre_date_dividends',
    '90_days_pre_date_dividends',
    
    '6_months_post_date_stock_price',
    '1_year_post_date_stock_price',
    '18_months_post_date_stock_price',
    
    '6_months_post_date_dividends',
    '1_year_post_date_dividends',
    '18_months_post_date_dividends'
    
]

list_percentage_columns = [
    '18_months_pre_date_total_return',
    '1_year_pre_date_total_return',
    '6_months_pre_date_total_return',
    '90_days_pre_date_total_return',
    'ownership_pecent_on_announcements',
    '6_months_post_date_total_return',
    '1_year_post_date_total_return',
    '18_months_post_date_total_return',
]

list_column_order = [
    
    # campaign
    'campaign_id',
    'campaign_announcement_date',
    'campaign_title',
    
    # campaign objective
    'campaign_objective_primary',
    'value_demand',
    'governance_demand',
    'activist_campaign_tactic',
    'activist_campaign_results',  # y variable
    
    # campaign objective details - board seats
    'total_number_of_board_seats',
    'number_of_board_seats_sought',
    'number_of_board_seats_gained',   # y variable
    'short_or_majority_or_full_slate',   # y variable
    
    # campaign objective details - proxy
    'proxy_proposal',
    'glass_lewis_support',
    'iss_support',
    'proxy_campaign_winner_or_result',  # y variable
    
    # activist
    'activist_id',
    'activist_name',
    'activist_group',
    
    # activist holding information
    'first_trade_date',
    'last_trade_date',
    'ownership_pecent_on_announcements',
    
    # company
    'company_id',
    'company_name',
    'sector',
    
    # comapny fundamentals
    'price_at_announcement',
    'ltm_eps_at_announcement',
    'earnings_yield_at_announcement',
    
    # company features
    'current_entity_status',
    'current_entity_detail',
    'public_before_or_after_campaign_announcement',
    'poison_pill_in_force_prior_to_announcement',
    'poison_pill_adopted_in_response_to_campaign',
    
    # dates
    '18_months_pre_announcement_date',
    '1_year_pre_announcement_date',
    '6_months_pre_announcement_date',
    '90_days_pre_announcement_date',
    '6_months_post_announcement_date',
    '1_year_post_announcement_date',
    '18_months_post_announcement_date',

    # company returns
    '18_months_pre_date_total_return',
    '1_year_pre_date_total_return',
    '6_months_pre_date_total_return',
    '90_days_pre_date_total_return',
    '6_months_post_date_total_return',  # y variable
    '1_year_post_date_total_return',  # y variable
    '18_months_post_date_total_return'  # y variable

]

# Read Raw Data

In [3]:
df_raw = pd.read_excel('../data/FactSet_Campaign v8.xlsx', skiprows=2, na_values=['', ' ', '_', '-', 'na', 'NA', 'n.a.'])

In [4]:
df_raw.iloc[0]

Campaign Symbol                                                                      0851399389C
Campaign Title                                 1-800-FLOWERS.COM, Inc. / GAMCO Asset Manageme...
Campaign Objective Primary                            13D Filer - No Publicly Disclosed Activism
Activist                                                            GAMCO Asset Management, Inc.
Activist ID                                                                             000KVL-E
Campaign Announce Date                                                                  20121004
18-Months Pre Announcement_Dates                                             2011-04-04 00:00:00
1-Year Pre Announcement_Dates                                                2011-10-04 00:00:00
6-Months Pre Announcement_Dates                                              2012-04-04 00:00:00
90-Days Pre Announcement_Dates                                               2012-07-06 00:00:00
Announcement Date_Dates       

# Clean Data

In [5]:
df_cleaning = (
    
    df_raw
    
    # rename 
    .rename(columns=clean_column_name)
    .rename(columns=dict_manual_renamings)
    
    # drop
    .drop(axis='columns', labels=list_drop_columns)
    
    # convert strings to dates based on format
    # note this fails silently for malformed dates for now
    .assign(campaign_announcement_date=lambda df: pd.to_datetime(df.campaign_announcement_date, format='%Y%m%d'))
    .assign(first_trade_date=lambda df: pd.to_datetime(df.first_trade_date, format='%Y-%m-%d %H:%M:%S'))
    .assign(last_trade_date=lambda df: pd.to_datetime(df.last_trade_date.astype(str), format='%m/%d/%Y', errors='coerce'))
    
    # extract company name and activist group from campaign title
    # note that what comes after the / can be a list of comma separated activist names, I call this activist group
    .assign(company_name=lambda df: df.campaign_title.str.split(' / ', n=1, expand=True)[0])
    .assign(activist_group=lambda df: df.campaign_title.str.split(' / ', n=1, expand=True)[1])
    
    # for categoricals, standardize to Title case
    .assign(sector=lambda df: df.sector.str.title())
    .assign(public_before_or_after_campaign_announcement=lambda df: df.public_before_or_after_campaign_announcement.str.title())
    .assign(current_entity_status=lambda df: df.current_entity_status.str.title())
    .assign(current_entity_detail=lambda df: df.current_entity_detail.str.title())
    
    # light features
    .assign(earnings_yield_at_announcement=lambda df: df.ltm_eps_at_announcement / df.price_at_announcement)
    
)

# from percentages to raw units
for percentage_column in list_percentage_columns:
    df_cleaning[percentage_column] = df_cleaning[percentage_column] / 100

# reorder
df_cleaning = (
    df_cleaning
    .loc[:, list_column_order]
    .sort_values(['campaign_id', 'campaign_announcement_date', 'campaign_title'])
)
    
df_clean = df_cleaning

In [6]:
df_clean.iloc[0]

campaign_id                                                                           0000054704C
campaign_announcement_date                                                    2007-09-24 00:00:00
campaign_title                                  Catalytica Energy Systems, Inc. / AWM Investme...
campaign_objective_primary                                         Vote/Activism Against a Merger
value_demand                                    Block Acquisition/Agitate for Lower Price (Sha...
governance_demand                                                                             NaN
activist_campaign_tactic                            Publicly Disclosed Letter to Board/Management
activist_campaign_results                       Campaign to vote against proposed merger with ...
total_number_of_board_seats                                                                     0
number_of_board_seats_sought                                                                    0
number_of_board_seat

In [7]:
df_clean.dtypes

campaign_id                                             object
campaign_announcement_date                      datetime64[ns]
campaign_title                                          object
campaign_objective_primary                              object
value_demand                                            object
governance_demand                                       object
activist_campaign_tactic                                object
activist_campaign_results                               object
total_number_of_board_seats                              int64
number_of_board_seats_sought                             int64
number_of_board_seats_gained                           float64
short_or_majority_or_full_slate                         object
proxy_proposal                                          object
glass_lewis_support                                     object
iss_support                                             object
proxy_campaign_winner_or_result                        

In [8]:
len(df_clean)

9571

# Write Clean Data

In [9]:
df_clean.to_csv('../data/clean_factset_campaign_data.csv', index=False)

# Check Data

# Campaigns

Keyed by `(campaign_id, activist_id, company_id)`.

In [10]:
df_clean.campaign_id.nunique()

9571

In [11]:
df_campaign = (
    df_clean
    .groupby('campaign_id')
    .last()
    .reset_index()
)

In [12]:
df_campaign.head(5)

,campaign_id,campaign_announcement_date,campaign_title,campaign_objective_primary,value_demand,governance_demand,activist_campaign_tactic,activist_campaign_results,total_number_of_board_seats,number_of_board_seats_sought,...,6_months_post_announcement_date,1_year_post_announcement_date,18_months_post_announcement_date,18_months_pre_date_total_return,1_year_pre_date_total_return,6_months_pre_date_total_return,90_days_pre_date_total_return,6_months_post_date_total_return,1_year_post_date_total_return,18_months_post_date_total_return
0,0000054704C,2007-09-24,"Catalytica Energy Systems, Inc. / AWM Investme...",Vote/Activism Against a Merger,Block Acquisition/Agitate for Lower Price (Sha...,NaN,Publicly Disclosed Letter to Board/Management,Campaign to vote against proposed merger with ...,0,0,...,2008-03-24,2008-09-24,2009-03-24,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0000396364C,2008-01-22,"Circuit City Stores, Inc. / Wattles Capital Ma...",Board Control,Review Strategic Alternatives,Remove Director(s),"Propose Binding Proposal, Publicly Disclosed L...",Proxy fight to remove and replace the board se...,12,5,...,2008-07-22,2009-01-22,2009-07-22,-0.007917,-0.007622,-0.006603,-0.004190,-0.004637,-0.009898,-0.009892
2,0000411278C,2012-05-29,Reading International Inc. / Capstone Equities...,Maximize Shareholder Value,"Breakup Company, Divest Assets/Divisions",Other Governance Enhancements,Publicly Disclosed Letter to Board/Management,"Capstone urged a breakup, saying that a sum-of...",0,0,...,2012-11-29,2013-05-29,2013-11-29,0.001575,0.002124,0.004033,0.002952,-0.000323,0.000459,0.002398
3,0000556550C,2008-03-24,"Coinstar, Inc. / Shamrock Partners Activist Va...",Board Representation,NaN,Other Governance Enhancements,"Nominate Slate of Directors, Letter to Stockho...",Proxy fight for 3 of 7 seats settled in exchan...,7,3,...,2008-09-24,2009-03-24,2009-09-24,0.000496,-0.000649,-0.001156,-0.000070,0.001723,0.000214,0.001033
4,0000719478C,2011-08-05,"Arch Chemicals, Inc. / GAMCO Investors",13D Filer - No Publicly Disclosed Activism,NaN,NaN,NaN,13D Filer - No Publicly Disclosed Activism,0,0,...,2012-02-05,2012-08-05,2013-02-05,0.006649,0.003445,0.003305,0.003133,NaN,NaN,NaN


In [13]:
df_campaign.groupby('campaign_objective_primary').campaign_id.count().sort_values(ascending=False).to_frame('count')

,count
campaign_objective_primary,
Board Representation,2131
Maximize Shareholder Value,2078
13D Filer - No Publicly Disclosed Activism,1256
Vote For a Stockholder Proposal,1009
Board Control,790
Vote/Activism Against a Merger,562
Hostile/Unsolicited Acquisition,556
Vote Against a Management Proposal,362
Enhance Corporate Governance,270


In [14]:
df_campaign.groupby('value_demand').campaign_id.count().sort_values(ascending=False).to_frame('count')

,count
value_demand,
Potential Acquisition (Friendly and Unfriendly),750
Review Strategic Alternatives,511
"Breakup Company, Divest Assets/Divisions",469
Block Merger/Agitate for Higher Price (Shareholder of Target),467
Return Cash via Dividends/Buybacks,443
Seek Sale/Merger/Liquidation,284
"Other Capital Structure Related, Increase Leverage, etc.",278
Fund specific: Realize Net Asset Value/Open-End a Closed-End Fund,259
Block Acquisition/Agitate for Lower Price (Shareholder of Acquirer),98


In [15]:
df_campaign.groupby('governance_demand').campaign_id.count().sort_values(ascending=False).to_frame('count')

,count
governance_demand,
Board Seats (activist group),1791
Other Governance Enhancements,814
Compensation Related Enhancements,662
Remove Director(s),379
Social/Environmental/Political Issues,356
Remove Takeover Defenses,297
Add Independent Directors,271
Remove Officer(s),77


In [16]:
df_campaign[[c for c in df_campaign if 'return' in c]].describe()

,18_months_pre_date_total_return,1_year_pre_date_total_return,6_months_pre_date_total_return,90_days_pre_date_total_return,6_months_post_date_total_return,1_year_post_date_total_return,18_months_post_date_total_return
count,8640.000000,8798.000000,8884.000000,8914.000000,8030.000000,7521.000000,7148.000000
mean,0.002285,0.001127,0.000887,0.000593,0.000484,0.000805,0.001159
std,0.082765,0.019311,0.022915,0.006367,0.005671,0.015886,0.021140
min,-0.009999,-0.009990,-0.009956,-0.009839,-0.010000,-0.010000,-0.010000
25%,-0.003241,-0.002777,-0.001752,-0.001118,-0.001577,-0.002567,-0.003121
50%,0.000143,0.000065,0.000245,0.000250,0.000230,0.000154,0.000146
75%,0.003345,0.002561,0.002092,0.001627,0.001797,0.002464,0.003027
max,7.568124,1.266316,2.051111,0.351111,0.265000,1.240000,1.590000


# Tactics

Keyed by `(campaign_id, activist_id, company_id, activist_campaign_tactic)`.

In [17]:
df_tactic = (
    df_clean
    .groupby('campaign_id')
    [
        'activist_id',
        'company_id',
        'activist_campaign_tactic'
    ]
    .last()
    .reset_index()
    .assign(activist_campaign_tactic=lambda df: df.activist_campaign_tactic.fillna('No or Unknown'))
    .assign(activist_campaign_tactic=lambda df: df.activist_campaign_tactic.str.split(', '))
    .explode('activist_campaign_tactic')
    .assign(activist_campaign_tactic_indicator=1)
)

In [18]:
df_tactic.head()

,campaign_id,activist_id,company_id,activist_campaign_tactic,activist_campaign_tactic_indicator
0,0000054704C,002HVP-E,0030TP-E,Publicly Disclosed Letter to Board/Management,1
1,0000396364C,006SZN-E,000DRZ-E,Propose Binding Proposal,1
1,0000396364C,006SZN-E,000DRZ-E,Publicly Disclosed Letter to Board/Management,1
1,0000396364C,006SZN-E,000DRZ-E,Nominate Slate of Directors,1
2,0000411278C,00DFB7-E,000DS9-E,Publicly Disclosed Letter to Board/Management,1


In [19]:
df_tactic.groupby('activist_campaign_tactic').campaign_id.count().sort_values(ascending=False).to_frame('count')

,count
activist_campaign_tactic,
No or Unknown,3538
Publicly Disclosed Letter to Board/Management,3362
Nominate Slate of Directors,2003
Letter to Stockholders,1796
Propose Precatory Proposal,1171
Unsolicited Offer,762
Propose Binding Proposal,548
Threaten Proxy Fight,504
Lawsuit,477


In [20]:
df_tactics_indicators = (
    pd.pivot_table(df_tactic_indicators, index=['campaign_id'], columns=['activist_campaign_tactic'], values='activist_campaign_tactic_indicator')
    .rename(columns=clean_column_name)
    .rename(columns=lambda c: 'used_' + c + '_tactic')
)

NameError: name 'df_tactic_indicators' is not defined

In [ ]:
df_tactics_indicators.head(10)

# Activists

Keyed by `(activist_id)`.

In [ ]:
df_clean.activist_id.nunique()

In [ ]:
df_activist = (
    df_clean
    .groupby('activist_id')
    ['activist_name', 'activist_group']
    .last()
    .reset_index()
)

df_activist.head(5)

In [ ]:
(
    pd.merge(
        df_activist,
        df_campaign.groupby('activist_id').campaign_id.count().to_frame('campaign_count'),
        how='left',
        on=['activist_id']
    )
    .sort_values(by='campaign_count', ascending=False)
    .head(10)
)

# Targets

Keyed by `(company_id)`.

In [ ]:
df_clean.company_id.nunique()

In [ ]:
df_company = (
    df_clean
    .groupby('company_id')
    [
        'company_name',
        'sector',
        'current_entity_status',
        'current_entity_detail'
    ]
    .last()
    .reset_index()
)

In [ ]:
df_company.head(10)